In [7]:
import os

from utils.util import *
from train_test import *

In [8]:
MIND_type = 'small'
data_path = "/datasets/"

train_news_file = os.path.join(data_path, 'mind_train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'mind_train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'mind_val', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'mind_val', r'behaviors.tsv')
knowledge_graph_file = os.path.join(data_path, 'mind_kg', r'wikidata-graph.tsv')
entity_embedding_file = os.path.join(data_path, 'mind_kg', r'entity2vecd100.vec')
relation_embedding_file = os.path.join(data_path, 'mind_kg', r'relation2vecd100.vec')

mind_url, mind_train_dataset, mind_dev_dataset, _ = get_mind_data_set(MIND_type)


In [9]:
import argparse
from parse_config import ConfigParser


parser = argparse.ArgumentParser(description='KRED')

parser.add_argument('-f')


parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')

config = ConfigParser.from_args(parser)

In [10]:
from utils.cleaner import *
config = cleaner(config)

OSError: [Errno 30] Read-only file system: '/datasets/mind_val/test_news.tsv'

In [4]:
entity2id_dict = entity_to_id(config, entities_news(config))

In [5]:
entity_embedding = []

In [6]:
entity_embedding.append(np.zeros(config['model']['entity_embedding_dim']))

In [4]:
def construct_embedding_mind(config, entity2id_dict, entity_embedding, entity2embedding_dict):
    print('constructing embedding ...')
    relation_embedding = []
    zero_array = np.zeros(config['model']['entity_embedding_dim'])
    relation_embedding.append(zero_array)
    id2entity_dict = {v: k for k, v in entity2id_dict.items()}
    with open(config['data']['entity_embedding'], 'r', encoding='utf-8') as fp_entity_embedding:
        i = 1
        for line in fp_entity_embedding:
            if i in id2entity_dict:
              linesplit = line.strip().split('\t')
              linesplit = [float(i) for i in linesplit]
              entity2embedding_dict[id2entity_dict[i]] = len(entity_embedding)
              entity_embedding.append(linesplit)
            i += 1
    with open(config['data']['relation_embedding'], 'r', encoding='utf-8') as fp_relation_embedding:
        for line in fp_relation_embedding:
            linesplit = line.strip().split('\t')
            linesplit = [float(i) for i in linesplit]
            relation_embedding.append(linesplit)
    return entity2embedding_dict, entity_embedding, relation_embedding

In [11]:
entity2embedding_dict = {}
entity2embedding_dict, entity_embedding, relation_embedding = construct_embedding_mind(config, entity2id_dict, entity_embedding, entity2embedding_dict)


constructing embedding ...


In [12]:
entity_adj, relation_adj = construct_adj_mind(config, entity2id_dict, entity2embedding_dict)

TypeError: construct_adj_mind() takes 1 positional argument but 3 were given

In [26]:
config["data"]["train_news"].split('/')[-1]

'news.tsv'

In [1]:
from utils.util import entity_to_id

In [8]:
entity2id_dict = entity_to_id(config, entities_news(config))

In [9]:
entity2id_dict

{'Q34433': 8,
 'Q41': 9,
 'Q1860': 17,
 'Q30': 21,
 'Q60': 23,
 'Q847017': 31,
 'Q183': 32,
 'Q2736': 33,
 'Q131524': 48,
 'Q12788174': 50,
 'Q142': 51,
 'Q298': 58,
 'Q2887': 62,
 'Q1321': 63,
 'Q155': 66,
 'Q15180': 77,
 'Q408': 81,
 'Q38': 95,
 'Q29': 104,
 'Q2807': 105,
 'Q336286': 113,
 'Q1362411': 116,
 'Q37226': 122,
 'Q213': 124,
 'Q90': 142,
 'Q11424': 145,
 'Q25089': 146,
 'Q19877770': 147,
 'Q160432': 148,
 'Q83287': 154,
 'Q1384': 162,
 'Q2256': 171,
 'Q145': 172,
 'Q62': 186,
 'Q5083': 187,
 'Q1085': 190,
 'Q175621': 197,
 'Q174': 231,
 'Q128581': 233,
 'Q337585': 234,
 'Q33': 238,
 'Q193592': 239,
 'Q9617': 249,
 'Q9592': 261,
 'Q611644': 262,
 'Q250867': 264,
 'Q31': 292,
 'Q924': 302,
 'Q641': 308,
 'Q490': 311,
 'Q12152': 323,
 'Q1461': 324,
 'Q928': 326,
 'Q275482': 332,
 'Q1781': 337,
 'Q797': 345,
 'Q860626': 361,
 'Q1146335': 362,
 'Q126399': 368,
 'Q639669': 384,
 'Q131578': 388,
 'Q85': 390,
 'Q79': 395,
 'Q282722': 401,
 'Q467': 411,
 'Q202466': 412,
 'Q144': 41

In [29]:
if not os.path.exists("tmp"):
    os.makedirs("tmp")
with open(config["data"]["train_news"]) as old, open('tmp/train_'+ config["data"]["train_news"].split('/')[-1], 'w+') as new:
    for line in old:
        newsid, vert, subvert, title, abstract, url, entity_info_title, entity_info_abstract = line.strip().split('\t')
        entity_info_title = json.loads(entity_info_title)
        for index, entity in enumerate(entity_info_title):
            if entity2id_dict.get(entity['WikidataId'], config["data"]["num_entity_embedding"]) > config["data"]["num_entity_embedding"]:
                del entity_info_title[index]
        entity_info_title = json.dumps(entity_info_title)
        entity_info_abstract = json.loads(entity_info_abstract)
        for index, entity in enumerate(entity_info_abstract):
            if entity2id_dict.get(entity['WikidataId'], config["data"]["num_entity_embedding"]) > config["data"]["num_entity_embedding"]:
                del entity_info_abstract[index]
        entity_info_abstract = json.dumps(entity_info_abstract)
        new_line = '\t'.join([newsid, vert, subvert, title, abstract, url, entity_info_title, entity_info_abstract] )
        new.write(new_line+ '\n')

In [30]:
with open(config["data"]["valid_news"]) as old, open('tmp/test_' + config["data"]["valid_news"].split('/')[-1], 'w+') as new:
    for line in old:
        newsid, vert, subvert, title, abstract, url, entity_info_title, entity_info_abstract = line.strip().split('\t')
        entity_info_title = json.loads(entity_info_title)
        for index, entity in enumerate(entity_info_title):
            if entity2id_dict.get(entity['WikidataId'], config["data"]["num_entity_embedding"]) > config["data"]["num_entity_embedding"]:
                del entity_info_title[index]
        entity_info_title = json.dumps(entity_info_title)
        entity_info_abstract = json.loads(entity_info_abstract)
        for index, entity in enumerate(entity_info_abstract):
            if entity2id_dict.get(entity['WikidataId'], config["data"]["num_entity_embedding"]) > config["data"]["num_entity_embedding"]:
                del entity_info_abstract[index]
        entity_info_abstract = json.dumps(entity_info_abstract)
        new_line = '\t'.join([newsid, vert, subvert, title, abstract, url, entity_info_title, entity_info_abstract] )
        new.write(new_line+ '\n')

In [31]:

src_test

'tmp/test_news.tsv'

In [32]:
import shutil
src_test = 'tmp/test_' + config["data"]["valid_news"].split('/')[-1]
dst_test = '/'.join(config["data"]["valid_news"].split('/')[:-1]) + '/'
src_test
shutil.copy(src_test, dst_test)


OSError: [Errno 30] Read-only file system: '/datasets/mind_val/test_news.tsv'